# Introduction to Natural Language Processing 2 Lab04

## Introduction (1 point)
---

In [171]:
%%capture
!python3 -m pip install datasets transformers bertopic

### 1. (1 point) Pick one of the datasets between `hate` and `offensive`, and justify your choice. Remember that it is for a commercial application.

We choose to focus on Hate because a company providing a plateform could get a lot more problem allowing hate speech (legally speaking) compared to offensive speech. Also a Hate speech is way more specific and easily detected than offensive speech due to the lack of irony for example.

## Evaluating the dataset (5 points)

### (1 point) Describe the dataset. Look at the splits, proportion of classes, and see what you can figure out by just looking at the text.

In [172]:
from datasets import load_dataset
ds_builder = load_dataset("tweet_eval", "hate")

ds_builder

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2970
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [173]:
ds_builder['train']['text'][:5]

['@user nice new signage. Are you not concerned by Beatlemania -style hysterical crowds crongregating on you…',
 'A woman who you fucked multiple times saying yo dick small is a compliment you know u hit that spot 😎',
 '@user @user real talk do you have eyes or were they gouged out by a rapefugee?',
 'your girlfriend lookin at me like a groupie in this bitch!',
 'Hysterical woman like @user']

In [174]:
ds_builder['train']['label'][:5] #Looking at the labels

[0, 1, 1, 1, 0]

In [175]:
### 2. Nous sommes sur de 3/4 de données utilisées pour le training, 1/4 pour le testing et 1000 données utilisé pour la validation. On remarque également que les labels sont binaire.

We have a ratio of 3/4 of datas use for training and 1/4 for testing. We also have 1000 datas used independently as a validation set. We observe that the labels are binary values.

### (3 points) Use BERTopic to extract the topics within the data, and the main topics within each class. Please, think about fixing the random seed.

In [176]:
%%capture

from bertopic import BERTopic
from umap import UMAP

umap_model = UMAP(random_state=42) #Fixed the seed to 42

docs_train = ds_builder['train']['text']
docs_test = ds_builder['test']['text']
docs_validation = ds_builder['validation']['text']

#Create a model for each split of the dataset to see their topics
topic_model_train = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topic_model_test = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)
topic_model_validation = BERTopic(embedding_model="all-MiniLM-L6-v2", umap_model=umap_model)

topics_train, probs_train = topic_model_train.fit_transform(docs_train)
topics_test, probs_test = topic_model_test.fit_transform(docs_test)
topics_validation, probs_validation = topic_model_validation.fit_transform(docs_validation)

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when i

In [177]:
topic_model_train.get_topic_info() #Topic infos of the train split

,Topic,Count,Name
0,-1,3633,-1_the_and_to_you
1,0,310,0_rape_women_woman_raped
2,1,264,1_hysterical_woman_women_most
3,2,256,2_bitch_cunt_whore_shut
4,3,245,3_refugees_refugee_withrefugees_welcome
...,...,...,...
112,111,11,111_english_education_teacher_school
113,112,11,112_choke_cock_suck_twerk
114,113,10,113_crisis_noise_causing_mass
115,114,10,114_instagram_snapchat_updates_app


In [178]:
topic_model_train.get_topic(0) #Topic of the train split

[('rape', 0.07122964043044226),
 ('women', 0.022804178061013234),
 ('woman', 0.015212168797409411),
 ('raped', 0.013807579309422856),
 ('sexual', 0.012555998630613177),
 ('assault', 0.012521458026238415),
 ('men', 0.012008916403522486),
 ('weinstein', 0.011048865306812832),
 ('metoo', 0.011033561113770799),
 ('about', 0.010035057073188624)]

In [179]:
topic_model_train.visualize_topics() #Intertopic distance map

In [180]:
topic_model_train.visualize_barchart()

In [181]:
topic_model_test.get_topic_info() #Topic infos of the test split

,Topic,Count,Name
0,-1,628,-1_bitch_you_hoe_my
1,0,1516,0_user_the_to_buildthatwall
2,1,100,1_cunt_bitch_me_that
3,2,94,2_shoes_bitch_shoe_buy
4,3,77,3_hoe_you_bitch_ass
5,4,69,4_whore_bitch_me_you
6,5,63,5_bitch_user_he_lmao
7,6,61,6_hoe_you_bitch_hoes
8,7,50,7_cunt_user_bitch_he
9,8,48,8_hoe_she_you_bitch


In [182]:
topic_model_test.get_topic(0) #Topic of the test split

[('user', 0.04145063326430683),
 ('the', 0.030509867430702005),
 ('to', 0.026760963640424212),
 ('buildthatwall', 0.024022953044614527),
 ('maga', 0.020874298671553403),
 ('of', 0.020610427189945175),
 ('in', 0.019560536961289416),
 ('is', 0.019541040535484007),
 ('and', 0.019169731063295375),
 ('are', 0.01817558004179158)]

In [183]:
topic_model_test.visualize_topics() #Intertopic distance map

In [184]:
topic_model_test.visualize_barchart()

In [185]:
topic_model_validation.get_topic_info() #Topic infos of the validation split

,Topic,Count,Name
0,-1,80,-1_you_user_the_to
1,0,494,0_to_the_user_in
2,1,148,1_you_bitch_pussy_fuck
3,2,99,2_hysterical_user_woman_you
4,3,92,3_rape_women_user_woman
5,4,43,4_skank_you_user_your
6,5,29,5_hysterical_booker_woman_kavanaugh
7,6,15,6_kitchen_stay_the_in


In [186]:
topic_model_validation.get_topic(0) #Topic of the validation split

[('to', 0.05468022717494824),
 ('the', 0.05108708883817766),
 ('user', 0.048243394720869415),
 ('in', 0.040804872400757655),
 ('of', 0.03819326926907268),
 ('and', 0.036247845715687),
 ('refugees', 0.03216761355792318),
 ('is', 0.030811430730069866),
 ('immigrant', 0.024985811796947362),
 ('for', 0.024693333974017018)]

In [187]:
topic_model_validation.visualize_topics() #Intertopic distance map

In [188]:
topic_model_validation.visualize_barchart()

In [189]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/config.json
Model config BertConfig {
  "_name_or_path": "/root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_all-MiniLM-L6-v2/pytorch_model.bin
All model checkpoint weights were used when i

### (1 point) What do you think about the results? How do you think it could impact a model trained on these data?

The topics of test split and validation split are very similar while there is more topics inside the train split they are still similar between themself. A model trained on those data would likely be stuck on specific topics and not be able to generalize due to the fact that a lot of those topics are similar.

## Evaluate a model (5 points)
---

In [190]:
%%capture
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


task='hate'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
data_collector = DataCollatorWithPadding(tokenizer=tokenizer)

labels = ds_builder['test']['label']

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

encoded_input = ds_builder.map(lambda batch: tokenizer(batch["text"], padding=True), batched=True, remove_columns=["text", "label"])

trainer = Trainer(
    model, #Our model
    data_collator=data_collector, #Data Collector object we defined
    tokenizer=tokenizer, #Tokenizer object we defined
)

output = trainer.predict(encoded_input['test'])

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file cardiffnlp/twitter-roberta-base-hate/config.json
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-hate",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": tru

### (2 points) Evaluate their model on the test split of the dataset you picked, using precision, recall, and F1-score.

In [191]:
import torch.nn.functional as F

print(output.predictions)

out = torch.tensor(output.predictions)

predict = F.softmax(out, dim=1)

predict_label = torch.argmax(predict, dim=1)

m = torch.sum(predict_label == torch.tensor(labels))

print(m/len(predict))

[[-1.0485643   1.0212997 ]
 [ 0.46829826 -0.5080502 ]
 [-0.8273667   0.5418106 ]
 ...
 [-0.5798488   0.522204  ]
 [-0.49178636  0.40129316]
 [-1.3835905   1.4396809 ]]
tensor(0.5768)


In [192]:
from sklearn.metrics import classification_report

print(classification_report(predict_label, labels))

              precision    recall  f1-score   support

           0       0.31      0.89      0.46       591
           1       0.95      0.50      0.65      2379

    accuracy                           0.58      2970
   macro avg       0.63      0.69      0.55      2970
weighted avg       0.82      0.58      0.61      2970



### (3 points) Extract the top 50 tweets your model is most confident about in the target class (offensive or hateful), the top 50 in the neutral class, and the top 50 your model is most uncertain about. Do you believe the model is doing a great job? For at least 2 tweets the model wrongly classified in your target class, try explaining what could have gone wrong.

In [193]:
predict_label_sorted_i = torch.argsort(torch.tensor([x[1] for x in predict]))[-50::]

predict_label_sorted = predict[predict_label_sorted_i[-50::]]

predict_label_sorted

tensor([[0.0142, 0.9858],
        [0.0141, 0.9859],
        [0.0141, 0.9859],
        [0.0141, 0.9859],
        [0.0141, 0.9859],
        [0.0140, 0.9860],
        [0.0140, 0.9860],
        [0.0140, 0.9860],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0139, 0.9861],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0138, 0.9862],
        [0.0137, 0.9863],
        [0.0137, 0.9863],
        [0.0137, 0.9863],
        [0.0136, 0.9864],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0135, 0.9865],
        [0.0134, 0.9866],
        [0.0133, 0.9867],
        [0.0133, 0.9867],
        [0.0133, 0.9867],
        [0.0132, 0.9868],
        [0.0131, 0.9869],
        [0.0131, 0.9869],
        [0.0130, 0.9870],
        [0.0130, 0.9870],
        [0.0129, 0.9871],
        [0.0129, 0.9871],
        [0.0127, 0.9873],
        [0.0126, 0.9874],
        [0.0

In [194]:
predict_label_sorted_i

tensor([1377,  428,  617, 1298, 2308,  107,  247, 1143,  230, 1750, 1060,  286,
        1827,  872, 2079, 2769,  379,  233, 2664,  137, 2327, 1580,  939, 2463,
          64, 2921, 1996,   98, 1380,  792, 1275,  727, 1482,  412, 1136,  535,
        2075,  831, 2367, 1368, 1052, 1645,  531,   12, 1972, 2774, 2052, 1109,
         582,  407])

In [195]:
# We display the text the model is the most confident about
most_confidents = [ds_builder['test']['text'][i] for i in predict_label_sorted_i]
most_confidents

['Illegal Alien CrimeAn overwhelming majority of the Illegal Alien crimes I post every single day, EVERY SINGLE DAY, are committed by dREAMER age criminals!These heinous monsters are who the Left describes as magnificent patriots.#BuildTheWall #DeportThemAll #KeepAmericaSafe',
 '@user @user @user tell ME-What could people who come through the BACK DOOR,bring to our country-Except 4 CRIME & DISEASE ??#BuildThatWallU Can share',
 'Is she admitting to a crime here? Sure sounds like it to me, traitor who should be in prison for life @user @user #BuildTheWall #EndDACA #BoycottNFL #BoycottNike #LockThemAllUp',
 'Enough! #BuildTheWall #DeportThemAll Send the criminals BACK to their country of origin (once we have a GREAT big WALL!)',
 '@user Please DO NOT listen to lame duck @user immigration/The Wall is way more important than tax 2.0! Stick to your instinct #BuildThatWall #BuildTheDamnWall #DeportThemAll',
 'WE AS A NATION DESERVE A REASONABLE EXPECTATION OF SAFETY IN OUR OWN COUNTRY! VOTE 

In [196]:
predict_label_sorted_i_neutral = torch.argsort(torch.tensor([x[0] for x in predict]))[-50::]

predict_label_sorted_neutral = predict[predict_label_sorted_i_neutral]

predict_label_sorted_neutral

tensor([[0.9353, 0.0647],
        [0.9354, 0.0646],
        [0.9377, 0.0623],
        [0.9380, 0.0620],
        [0.9388, 0.0612],
        [0.9400, 0.0600],
        [0.9400, 0.0600],
        [0.9406, 0.0594],
        [0.9408, 0.0592],
        [0.9412, 0.0588],
        [0.9415, 0.0585],
        [0.9428, 0.0572],
        [0.9433, 0.0567],
        [0.9435, 0.0565],
        [0.9436, 0.0564],
        [0.9436, 0.0564],
        [0.9443, 0.0557],
        [0.9456, 0.0544],
        [0.9456, 0.0544],
        [0.9470, 0.0530],
        [0.9487, 0.0513],
        [0.9508, 0.0492],
        [0.9541, 0.0459],
        [0.9543, 0.0457],
        [0.9550, 0.0450],
        [0.9553, 0.0447],
        [0.9588, 0.0412],
        [0.9590, 0.0410],
        [0.9592, 0.0408],
        [0.9598, 0.0402],
        [0.9621, 0.0379],
        [0.9625, 0.0375],
        [0.9629, 0.0371],
        [0.9630, 0.0370],
        [0.9646, 0.0354],
        [0.9656, 0.0344],
        [0.9667, 0.0333],
        [0.9714, 0.0286],
        [0.9

In [197]:
most_neutral = [ds_builder['test']['text'][i] for i in predict_label_sorted_i_neutral]

most_neutral

["I LOVE WATCHING YOUTUBE VIDEOS WHERE THEY CALL OUT YOUTUBERS OR WHOEVER. I DEADASS LOVE CALL OUT CULTURE, IT'S PETTY BUT BITCH AND?",
 "The Atlantic takes a close look at how programmes for refugees in America are changing for the worse under President Trump's administration. #USA #world #Refugees #RefugeesWelcome #refugeesNOTwelcome #symmitree #4change",
 "I'm glad I have friends who call each other queens and goddesses not bitch or whore.     #goalsreached",
 'Force yourself to love yourself! Love it!! Be it!! Feel it!! Slay it!! Get into your kunt!!!',
 '8. I REMEMBER LIVE DMING U DWP. ur a g and i hope yr 11 treats u right. REMEMBER MY ADIVCE U BITCH ASS CUNT. ilysm!!',
 'BITCH MINSEOK CENTER IS SO DAMN BEAUTIFUL A WHOLE MAIN DANCER',
 'i rlly rlly rlly rlly rlly wanna buy those puma shoes but bITCH I JUST BOUGHT AN ALBUM AND I HAVE LIKE 50 PESOS IN MY WALLET',
 "Trump is a wicked son of a bitch. Can't get his way, so why not hurt everyone. Whoever said he was the best 4🇺🇸was…",


In [198]:
predict_label_sorted_i_uncertain = torch.argsort(torch.tensor([abs(x[0] - x[1]) for x in predict]))[:50:]

predict_label_sorted_uncertain = predict[predict_label_sorted_i_uncertain]

predict_label_sorted_uncertain

tensor([[0.4993, 0.5007],
        [0.5010, 0.4990],
        [0.4985, 0.5015],
        [0.4982, 0.5018],
        [0.4981, 0.5019],
        [0.5019, 0.4981],
        [0.5024, 0.4976],
        [0.5025, 0.4975],
        [0.4965, 0.5035],
        [0.4963, 0.5037],
        [0.5038, 0.4962],
        [0.5039, 0.4961],
        [0.4958, 0.5042],
        [0.5043, 0.4957],
        [0.5052, 0.4948],
        [0.4947, 0.5053],
        [0.4946, 0.5054],
        [0.4944, 0.5056],
        [0.5059, 0.4941],
        [0.4937, 0.5063],
        [0.5065, 0.4935],
        [0.5065, 0.4935],
        [0.5069, 0.4931],
        [0.5073, 0.4927],
        [0.5075, 0.4925],
        [0.5078, 0.4922],
        [0.5080, 0.4920],
        [0.4918, 0.5082],
        [0.4916, 0.5084],
        [0.4914, 0.5086],
        [0.5086, 0.4914],
        [0.4913, 0.5087],
        [0.4912, 0.5088],
        [0.4907, 0.5093],
        [0.4897, 0.5103],
        [0.4892, 0.5108],
        [0.5108, 0.4892],
        [0.4891, 0.5109],
        [0.5

In [199]:
most_uncertain = [ds_builder['test']['text'][i] for i in predict_label_sorted_i_uncertain]

most_uncertain

['@user @user . And until Trump made it an issue also the aim of most Politicians - cheap labor & votes for the elites. Most still refuse to support reform. #StopImmigration',
 "y'all ... I came to the gym all excited to see my trainer and he says why are you wearing chanclas? A bitch forgot to switch shoes. 😒☹️",
 "Okay you know what you Chabby fans can't bitch that Gabi is over everyday when that whore Abigail has been on for 10 days straight #Days",
 'Boy, bye😌 "@diorwhore Delete his number and put a face mask on bitch it\'s recovery szn"',
 'This is a couple who just moved on in to Australia and thought if we behave ourselves and involve ourselves in the Community, and have a child here then we will be allowed to stay.Sorry - if that got you in we would be swamped with MILLIONS just like you.#SendThemBack.',
 'Love that u used WOMEN instead of bitch.',
 '@user @user @user Dude your not showing your face so why should I bitch',
 'im that bitch who goes thru the whole comment section

## **Annotate data (7 points)**

In [200]:
!wget https://raw.githubusercontent.com/mvonwyl/epita/master/NLP/2022/08/tweets.json

--2022-11-23 19:22:23--  https://raw.githubusercontent.com/mvonwyl/epita/master/NLP/2022/08/tweets.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2107788 (2.0M) [text/plain]
Saving to: ‘tweets.json.3’

tweets.json.3       100%[===================>]   2.01M  --.-KB/s    in 0.02s   

2022-11-23 19:22:23 (89.4 MB/s) - ‘tweets.json.3’ saved [2107788/2107788]



In [201]:
json_data = load_dataset('json', data_files='tweets.json')

  0%|          | 0/1 [00:00<?, ?it/s]

In [202]:
json_data

DatasetDict({
    train: Dataset({
        features: ['id', 'id_str', 'text', 'lang', 'created_at'],
        num_rows: 10000
    })
})

In [203]:
json_data['train'][:3]

{'id': [1410492618790817793, 1410492618769780742, 1410492618790686720],
 'id_str': ['1410492618790817793',
  '1410492618769780742',
  '1410492618790686720'],
 'text': ['YOU BETTER SUCK HIS DICK KOZY I SEE YOU WITH KNUCKLES GET EM GYAAAAL',
  'I still canr believe it.😭😭😭😭😭',
  'You should raise the webform....how would they know then that you completed ur medicals'],
 'lang': ['en', 'en', 'en'],
 'created_at': ['Thu Jul 01 06:57:00 +0000 2021',
  'Thu Jul 01 06:57:00 +0000 2021',
  'Thu Jul 01 06:57:00 +0000 2021']}

In [204]:
encoded_input = json_data.map(lambda batch: tokenizer(batch["text"], padding=True), batched=True, remove_columns=["text", "created_at"])

output = trainer.predict(encoded_input['train'])

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: lang, id, id_str. If lang, id, id_str are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [205]:
print(output.predictions)

out = torch.tensor(output.predictions)

predict = F.softmax(out, dim=1)

predict_label = torch.argmax(predict, dim=1)

[[-0.33326095  0.07276524]
 [ 1.74078    -1.485999  ]
 [ 1.6898811  -1.6296374 ]
 ...
 [ 1.608491   -1.5347848 ]
 [ 1.2212311  -1.1041771 ]
 [ 1.5878689  -1.4942871 ]]


In [206]:
predict_label_sorted_i = torch.argsort(torch.tensor([x[1] for x in predict]))[-20::]

predict_label_sorted = predict[predict_label_sorted_i]

predict_label_sorted

tensor([[0.1977, 0.8023],
        [0.1929, 0.8071],
        [0.1929, 0.8071],
        [0.1725, 0.8275],
        [0.1569, 0.8431],
        [0.1488, 0.8512],
        [0.1486, 0.8514],
        [0.1457, 0.8543],
        [0.1359, 0.8641],
        [0.1318, 0.8682],
        [0.1250, 0.8750],
        [0.1206, 0.8794],
        [0.1179, 0.8821],
        [0.1010, 0.8990],
        [0.0893, 0.9107],
        [0.0810, 0.9190],
        [0.0727, 0.9273],
        [0.0695, 0.9305],
        [0.0679, 0.9321],
        [0.0608, 0.9392]])

In [207]:
hateful_20 = [json_data['train']['text'][i] for i in predict_label_sorted_i]

hateful_20

['The law of NATURE? Where there is a need for a FEMALE and a MALE to procreate. That might be why...',
 'This issa lie! Black women don’t cheat.',
 'Suck on this mf',
 'The sooner you realise vagina runs the world the better!!',
 'DAMN U DIDNT HAVE TO BE THAT AGGRESSIVE WHORE',
 'Stop having side chicks, this creates more problems for you...solve your issues at home',
 'This is why women don’t like #you',
 'Only a woman that is interested for material things and money says she agrees with her man doing anything he wants. If she loves you, she gonna slap you and broke your head till you make it right!',
 'can’t keep a lil hoe',
 'Better body then most woman on here',
 'One thing bout me we all gon laugh in this bitch🤣',
 'I wish more ladies would be self obsessed! So hard to get people to see the worth in themselves!',
 'Returning bitches to they last owner',
 "You're a little bitch",
 'How many bitches you gone have wearing the same number ?😂',
 'Lmaoooo bitches b moving different',
 

# Annotate data (7 points)
---

### 1. (1 point) Extract about 100 tweets containing at least 20% of your target class (offensive/hateful), from the 10K tweets provided. You can use the pretrained model to help you find tweets in the target class.

In [208]:
import random
random.seed(42)

x = 0
random_text = set()
n = len(predict)

while x < 80:
    i = random.randint(0, n)
    if i in predict_label_sorted_i or json_data['train']['text'][i] in random_text:
      continue

    random_text.add(json_data['train']['text'][i])
    x += 1

In [209]:
tweet_100 = np.concatenate([hateful_20 ,list(random_text)])

tweet_100

array(['The law of NATURE? Where there is a need for a FEMALE and a MALE to procreate. That might be why...',
       'This issa lie! Black women don’t cheat.', 'Suck on this mf',
       'The sooner you realise vagina runs the world the better!!',
       'DAMN U DIDNT HAVE TO BE THAT AGGRESSIVE WHORE',
       'Stop having side chicks, this creates more problems for you...solve your issues at home',
       'This is why women don’t like #you',
       'Only a woman that is interested for material things and money says she agrees with her man doing anything he wants. If she loves you, she gonna slap you and broke your head till you make it right!',
       'can’t keep a lil hoe', 'Better body then most woman on here',
       'One thing bout me we all gon laugh in this bitch🤣',
       'I wish more ladies would be self obsessed! So hard to get people to see the worth in themselves!',
       'Returning bitches to they last owner', "You're a little bitch",
       'How many bitches you gone have 

In [210]:
import pandas as pd

DF = pd.DataFrame(tweet_100)

DF.to_csv("tweet_100.csv", index=False)

### 2. (3 points) Altogether, write down an annotation guildeline (which should be at least 2/3 of a page long).
- **What does the target class look like?** 

We choose to work on the 'hate' dataset. Based on this defintion : *In common language, “hate speech” refers to offensive discourse targeting a group or an individual based on inherent characteristics (such as race, religion or gender) and that may threaten social peace* we choose to apply the following rules :    
  - key theme such as race, religion or gender in a context of a hateful text
  - context that threaten social peace

- **Any examples you could provide for ambiguous cases?**

*Example 1: 'You’re gonna be waiting a LOOONG time'*, this one as no context so we cannot tell if it's a threat or not.

*Example 2: 'Lmaoooo bitches b moving different'*, this also have no context, we cannot see if it's an insult or a figure of speech for an undetermined context.

- **Keep "Can't tell / not annotable" class. Make sure you document what this class mean in your guideline.**

The "Can't tell/not annotable" class is basically the class where we put all the text without context and/or that are too small to have enough information on the meaning.


### 3. (1 point) Every person in your group is going to annotate these tweets separately. So if you are 4, annotate them 4 times.
- **Typically, create a Google sheet or an excel document, one tab per person, in each tab one column for the text, and annother on the class.**

### 4. (2 point) Evaluate your inter-annotaor agreement using Cohen Kappa (if you are 2) or Fleiss Kappa.
- **If, like your teacher, you have issues making the NLTK implementation work on the latest version of python (3.10+), you can use the scikit-learn implementation of Cohen Kappa, and compute a matrix by pair of annotators.**
- **What does the score mean? Are you doing a good job annotating the data and, if not, why?**

In [211]:
!wget 'https://raw.githubusercontent.com/Miolith/nlp2-project/master/tweet_annotation_hate.csv')

from sklearn.metrics import cohen_kappa_score

df = pd.read_csv("tweet_annotation_hate.csv")

h_encoded = pd.get_dummies(df, columns=["Nelson","Karen","Romain"])

var = h_encoded[["Nelson_0","Nelson_1"]].values + h_encoded[["Karen_0","Karen_1"]].values + h_encoded[["Romain_0","Romain_1"]].values

/bin/bash: -c: line 0: syntax error near unexpected token `)'
/bin/bash: -c: line 0: `wget 'https://raw.githubusercontent.com/Miolith/nlp2-project/master/tweet_annotation_hate.csv')'


In [212]:
from statsmodels.stats import inter_rater as irr

kappa_score = irr.fleiss_kappa(var, method='fleiss')

print("Fleiss Kappa score :", kappa_score)

Fleiss Kappa score : 0.7560777298967392


Being between 0.6 < x < 0.8 is considered a "substantial agreement"

## Theoritical questions (16 points)
---

### (2 point) What is the purpose of subword tokenization used by transformer models?
- **Part of the answer is in the first part of the course (lesson 2).**

Subword Tokenization is used to avoid out of vocabulary issue such as word that aren't in the vocabulary of the train dataset. It allows us to encode word that weren't in the training set.

- **What is the effect on the vocabulary size?**

Due to this subword tokenization we reduce the size of the vocabulary to a reasonable size for the model to handle.

- **How does it impact out-of-vocabulary words (words which are not in the training data, but appear in the test data, or production environment)?**

We choose to split the rarely used words into subwords and not the oftenly used one. This allows us to handle out of vocabulary words

### (2 point) When building an encoder-decoder model using an RNN, what is the purpose of adding attention?
- **What problem are we trying to solve?**

We are trying to solve exploding and vanishing gradient issue. Furthermore RNN are known to be heavily bias toward the end of the model.

- **How does attention solve the problem?**

It connect every hidden state of the RNN together allowing us to better take in consideration the information of the input sequence.

### (2 point) In a transformer model what is the multihead attention used for?
- **What are we trying to achieve with self-attention?**

With self attention we connect every token of the input sequence allowing the model to have a better understanding of the context of word inside the sequence. It also allow better retropropagation due to the interconnectivity of the tokens.

- **Why do we use muliple head instead of one?**

Multiple head allow us to learn multiple type of relation between words.

### (2 point) In a transformer model, what is the purpose of positional embedding?

The positional embedding allow us to keep the informations on the order of tokens of the sequence.

- **What would be the problem if we didn't use it?**

We loose the information of the order of the words using attention, to keep an information about this we need to encode that information using the positonal embedding.

### (2 point) What are the are the purpose of benchmarks?
- **And are they reliable? Why?**

Benchmarks are metrics used to evaluate performance. So their reliability depends on the context of the problem and what kind of benchmarks we choose to use for wich problems.

### (4 points) What are the differences between BERT and GPT?

The main difference between BERT and GPT is that BERT is a bidirectional model, while GPT is unidirectional. This means that BERT can look at the entire sequence of words before making a prediction, while GPT can only look at the words before the current word. This is why BERT is better at understanding the context of a sentence, while GPT is better at generating text.

- **What kind of transformer-based model are they?**

BERT is a bidirectional model with an Encoder-only architecture, while GPT is unidirectional with an Decoder-only architecture.

- **How are they pretrained?**

BERT and GPT are pre-trained on a large corpus of text. BERT is pre-trained on the entire Wikipedia corpus, while GPT is pre-trained on the entire web text corpus.

- **How are they fine-tuned?**

GPT can be fine-tuned by adjusting the learning rate, number of training iterations, and other parameters to minimize the error between the predicted outputs and the actual labels.
BERT is fine-tuned by using a technique called masked language modeling. This technique involves randomly masking a portion of the input text and then training the model to predict the masked token.
It is done by adding a simple classification layer on top of the BERT model.

### (2 points) How are zero-shot and few-shots learning different from fine-tuning?

Fine tuning, zero-shot and few-shot learning are all ways of training a model on a specific task. Fine tuning is when you train a model on a specific task using the weights from a pre-trained model. Zero-shot learning is when you train a model on a specific task without using the weights from a pre-trained model. Few-shot learning is when you train a model on a specific task using a small amount of data.

- **How do fine-tuning, zero-shot, and few-shot learning affect the model's weights?**

Fine tuning modify the model's weights, zero-shot and few-shot learning do not update the weights because they only use the models

### (2 point) In a few paragraphs, explain how the triplet loss is used to train a bi-encoder model for semantic similarity?

The triplet loss is used to train a bi-encoder model for semantic similarity by maximizing the similarity between the two encodings of a given triplet. A triplet is composed of a query encoding, a positive encoding, and a negative encoding. The loss function is based on the distance between the query and positive encoder, and the distance between the query and negative encoder. The goal is to minimize the distance between the query and positive encoder, and maximize the distance between the query and negative encoder.

### (2 point) What is the purpose of using an Approximate Nearest Neighbour method to speed up search?

The purpose of using an Approximate Nearest Neighbour method to speed up search is to find the nearest neighbour of a query in a large dataset. It is used to reduce the dimensionality of data. This is done by creating a new data set that is a subset of the original data set. The new data set is then used to train a model.

- **What does it really reduce?**

The Approximate Nearest Neighbour method reduces the number of comparisons that need to be made to find the nearest neighbour of a query in a large dataset.